In [5]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import math
import random
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import shutil

# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
print(f"device : {device}")

device : mps


In [9]:
data_path = '/Users/kweonminseong/Documents/git/Kaggle-Notebooks/input/aerial-cactus-identification/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

train_path = '/Users/kweonminseong/Documents/git/Kaggle-Notebooks/input/aerial-cactus-identification/train/'
test_path = '/Users/kweonminseong/Documents/git/Kaggle-Notebooks/input/aerial-cactus-identification/test/'

In [10]:
train, valid = train_test_split(labels, test_size=0.1, stratify=labels['has_cactus'], random_state=50)

In [11]:
class ImageDataset(Dataset):
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__()
        self.df=df
        self.img_dir=img_dir
        self.transform=transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.df.iloc[idx, 1]

        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [16]:
transform = transforms.ToTensor()

dataset_train = ImageDataset(df=train, img_dir=train_path, transform=transform)
dataset_valid = ImageDataset(df=valid, img_dir=train_path, transform=transform)

loader_train = DataLoader(dataset=dataset_train, batch_size=32, shuffle=True)
loader_valid = DataLoader(dataset=dataset_valid, batch_size=32, shuffle=False)

In [17]:
class Model(nn.Module):
    # 신경망 계층 정의
    def __init__(self):
        super().__init__() # 상속받은 nn.Module의 __init__() 메서드 호출

        # 첫 번째 합성곱 계층
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32,
                               kernel_size=3, padding=2)
        # 두 번째 합성곱 계층
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64,
                               kernel_size=3, padding=2)
        # 최대 풀링 계층
        self.max_pool = nn.MaxPool2d(kernel_size=2)
        # 평균 풀링 계층
        self.avg_pool = nn.AvgPool2d(kernel_size=2)
        # 전결합 계층
        self.fc = nn.Linear(in_features=64 * 4 * 4, out_features=2)

    # 순전파 출력 정의
    def forward(self, x):
        x = self.max_pool(F.relu(self.conv1(x)))
        x = self.max_pool(F.relu(self.conv2(x)))
        x = self.avg_pool(x)
        x = x.view(-1, 64 * 4 * 4) # 평탄화
        x = self.fc(x)
        return x

In [18]:
model = Model().to(device)
model

Model(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (avg_pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=2, bias=True)
)

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [20]:
epochs = 100

for epoch in range(epochs):
    epoch_loss = 0 # epoch별 손실값 초기화

    # '반복 횟수' 만큼 반복
    for images, labels in loader_train:
        # img, labels data mini batch 할당
        images = images.to(device)
        labels = labels.to(device)

        # optimizer 내의 기울기 초기화
        optimizer.zero_grad()
        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 손실 함수를 활용해 outputs와 labels 의 손실값 계산
        loss = criterion(outputs, labels)
        # 현재 배치에서의 손실 추가
        epoch_loss += loss.item()
        # 역전파 수행
        loss.backward()
        # 가중치 갱신
        optimizer.step()

    # 훈련 데이터 손실값 출력
    print(f'에폭 [{epoch+1}/{epochs}] - 손실값: {epoch_loss/len(loader_train):.4f}')

에폭 [1/100] - 손실값: 0.5202
에폭 [2/100] - 손실값: 0.3492
에폭 [3/100] - 손실값: 0.2544
에폭 [4/100] - 손실값: 0.2097
에폭 [5/100] - 손실값: 0.1801
에폭 [6/100] - 손실값: 0.1651
에폭 [7/100] - 손실값: 0.1513
에폭 [8/100] - 손실값: 0.1389
에폭 [9/100] - 손실값: 0.1359
에폭 [10/100] - 손실값: 0.1246
에폭 [11/100] - 손실값: 0.1217
에폭 [12/100] - 손실값: 0.1186
에폭 [13/100] - 손실값: 0.1105
에폭 [14/100] - 손실값: 0.1109
에폭 [15/100] - 손실값: 0.1081
에폭 [16/100] - 손실값: 0.1055
에폭 [17/100] - 손실값: 0.1054
에폭 [18/100] - 손실값: 0.1029
에폭 [19/100] - 손실값: 0.1020
에폭 [20/100] - 손실값: 0.0999
에폭 [21/100] - 손실값: 0.0972
에폭 [22/100] - 손실값: 0.0973
에폭 [23/100] - 손실값: 0.0973
에폭 [24/100] - 손실값: 0.0946
에폭 [25/100] - 손실값: 0.0927
에폭 [26/100] - 손실값: 0.0925
에폭 [27/100] - 손실값: 0.0921
에폭 [28/100] - 손실값: 0.0902
에폭 [29/100] - 손실값: 0.0901
에폭 [30/100] - 손실값: 0.0898
에폭 [31/100] - 손실값: 0.0880
에폭 [32/100] - 손실값: 0.0871
에폭 [33/100] - 손실값: 0.0874
에폭 [34/100] - 손실값: 0.0841
에폭 [35/100] - 손실값: 0.0843
에폭 [36/100] - 손실값: 0.0821
에폭 [37/100] - 손실값: 0.0833
에폭 [38/100] - 손실값: 0.0809
에폭 [39/100] - 손실값: 0.

In [21]:
# 모델 검증
true_list = []
preds_list = []

model.eval() # 모델을 평가 상태로 설정

with torch.no_grad(): # 기울기 계산 비활성화
    for images, labels in loader_valid:
        # 이미지, 레이블 데이터 미니배치를 장비에 할당
        images = images.to(device)
        labels = labels.to(device)

        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1] # 예측 확률
        true = labels.cpu() # 실제값
        # 예측 확률과 실제값을 리스트에 추가
        preds_list.extend(preds)
        true_list.extend(true)

# 검증 데이터 ROC AUC 점수 계산
print(f'검증 데이터 ROC AUC : {roc_auc_score(true_list, preds_list):.4f}')

검증 데이터 ROC AUC : 0.9968


In [22]:
dataset_test = ImageDataset(df=submission, img_dir=test_path, transform=transform)
loader_test = DataLoader(dataset=dataset_test, batch_size=32, shuffle=False)

model.eval() # 모델을 평가 상태로 설정

preds = [] # 타깃 예측값 저장용 리스트 초기화

with torch.no_grad(): # 기울기 계산 비활성화
    for images, _ in loader_test:
        # 이미지 데이터 미니배치를 장비에 할당
        images = images.to(device)

        # 순전파 : 이미지 데이터를 신경망 모델의 입력값으로 사용해 출력값 계산
        outputs = model(images)
        # 타깃값이 1일 확률(예측값)
        preds_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist()
        # preds에 preds_part 이어붙이기
        preds.extend(preds_part)

In [23]:
submission['has_cactus'] = preds
submission.to_csv('submission.csv', index=False)